In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import os
from os.path import join
from datetime import datetime

import plotly.express as px

pd.options.mode.chained_assignment = None

In [2]:
def format_prezzi(df):
    df['Data'] = pd.to_datetime(df.Data.astype(str) + (df.Ora-1).astype(str), format='%Y%m%d%H')
    df.drop(['Mercato', 'Ora'], axis=1, inplace=True)
    for col in df.columns:
        if col not in ['Data', 'Ora']:
            df[col] = df[col].str.replace(',', '.').astype(float)


In [3]:
market = 'MGP'
dataset = 'DomandaOfferta'
freq = 'daily'
date_range = pd.date_range(start='2023-11-01', end='2023-11-30')

In [4]:
df_list = []

for date in tqdm(date_range):
    
    date = date.strftime('%Y%m%d')
    file_path = join('data', market + dataset, freq, str(date) + market + dataset + '.xml')
    
    if not(os.path.exists(file_path)):
        print(f"{file_path} not found")
    else:
        df = pd.read_xml(file_path, xpath=f'./{dataset}')
    df_list.append(df)

ts_df = pd.concat(df_list, ignore_index=True)
#format(df)
ts_df

  0%|          | 0/30 [00:00<?, ?it/s]

,Data,Ora,Mercato,ZonaMercato,PrezzoZonale,Quantita,Prezzo,Tipo
0,20231101,1,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,64.97417,0.001,800.00,OFF
1,20231101,1,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,64.97417,7.500,700.00,OFF
2,20231101,1,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,64.97417,0.003,632.00,OFF
3,20231101,1,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,64.97417,62.750,499.00,OFF
4,20231101,1,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,64.97417,100.000,353.29,OFF
...,...,...,...,...,...,...,...,...
242502,20231130,24,MGP,CNOR;NORD;AUST;CORS;FRAN;SLOV;SVIZ;COUP;,115.61000,0.100,60.00,BID
242503,20231130,24,MGP,CNOR;NORD;AUST;CORS;FRAN;SLOV;SVIZ;COUP;,115.61000,4.900,11.00,BID
242504,20231130,24,MGP,CNOR;NORD;AUST;CORS;FRAN;SLOV;SVIZ;COUP;,115.61000,0.002,1.00,BID
242505,20231130,24,MGP,CNOR;NORD;AUST;CORS;FRAN;SLOV;SVIZ;COUP;,115.61000,1.911,0.00,BID


In [5]:
ts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242507 entries, 0 to 242506
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Data          242507 non-null  int64  
 1   Ora           242507 non-null  int64  
 2   Mercato       242507 non-null  object 
 3   ZonaMercato   242507 non-null  object 
 4   PrezzoZonale  242507 non-null  float64
 5   Quantita      242507 non-null  float64
 6   Prezzo        242507 non-null  float64
 7   Tipo          242507 non-null  object 
dtypes: float64(3), int64(2), object(3)
memory usage: 14.8+ MB


In [6]:
ts_df.Tipo.value_counts()

Tipo
OFF    160181
BID     82326
Name: count, dtype: int64

In [7]:
ts_df.ZonaMercato.value_counts()

ZonaMercato
CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CORS;FRAN;GREC;SLOV;SVIZ;MALT;COUP;MONT;    170232
CALA;CNOR;CSUD;SARD;SICI;SUD;COAC;CORS;GREC;MALT;MONT;                                   15187
NORD;AUST;FRAN;SLOV;SVIZ;COUP;                                                           14725
CNOR;NORD;AUST;CORS;FRAN;SLOV;SVIZ;COUP;                                                 13137
CALA;CSUD;SARD;SICI;SUD;COAC;GREC;MALT;MONT;                                             10136
CNOR;CSUD;NORD;SARD;AUST;COAC;CORS;FRAN;SLOV;SVIZ;COUP;MONT;                              5796
CALA;SICI;SUD;GREC;MALT;                                                                  4371
CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CORS;FRAN;GREC;SLOV;SVIZ;COUP;MONT;           1898
CSUD;SARD;COAC;MONT;                                                                      1419
CNOR;NORD;AUST;FRAN;SLOV;SVIZ;COUP;                                                       1205
CALA;CNOR;CSUD;NORD;SICI;SUD;AUST;FRAN

In [30]:
# df = ts_df.loc[(ts_df.Data==20231130)&(ts_df.Ora==9)] # Example of a time when congestion occurs
df = ts_df.loc[(ts_df.Data==20231130)&(ts_df.Ora==4)] # Example of a time when no congestion occurs

In [31]:
df[['PrezzoZonale', 'ZonaMercato']].drop_duplicates()

,PrezzoZonale,ZonaMercato
232677,107.16,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...


In [32]:
df.loc[:,'_sorting'] = df.Prezzo
df.loc[df.Tipo == 'BID', '_sorting'] = -df._sorting
df.sort_values(by=['ZonaMercato', 'Tipo', '_sorting'], inplace=True)
df['_ZonaTipo'] = df['ZonaMercato'] + df['Tipo'] 
df['cumQuantita'] = df.groupby('_ZonaTipo')['Quantita'].cumsum()
# df['cumQuantita'] = df.groupby(['ZonaMercato', 'Tipo'])['Quantita'].cumsum()
df

,Data,Ora,Mercato,ZonaMercato,PrezzoZonale,Quantita,Prezzo,Tipo,_sorting,_ZonaTipo,cumQuantita
232908,20231130,4,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,107.16,15084.490,4000.0,BID,-4000.0,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,15084.490
232909,20231130,4,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,107.16,2.499,3999.0,BID,-3999.0,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,15086.989
232910,20231130,4,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,107.16,24.630,3950.0,BID,-3950.0,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,15111.619
232911,20231130,4,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,107.16,45.560,3500.0,BID,-3500.0,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,15157.179
232912,20231130,4,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,107.16,4542.895,3000.0,BID,-3000.0,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,19700.074
...,...,...,...,...,...,...,...,...,...,...,...
232681,20231130,4,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,107.16,103.000,344.1,OFF,344.1,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,50499.772
232680,20231130,4,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,107.16,20.000,400.0,OFF,400.0,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,50519.772
232679,20231130,4,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,107.16,0.003,632.0,OFF,632.0,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,50519.775
232678,20231130,4,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,107.16,7.500,700.0,OFF,700.0,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,50527.275


In [33]:
fig = px.line(
    df,
    x = 'cumQuantita',
    y = 'Prezzo',
    line_group = '_ZonaTipo',
    color = 'ZonaMercato',
    title = 'Supply and Demand Curves',
    height = 800,
    width = 1200,
    line_shape = 'hv'
)

fig.update_xaxes(title_text='Cumulative volume (MWh)')
fig.update_yaxes(title_text='Price (€/MWh)')
fig.update_layout(legend=dict(orientation="h", yanchor="top", y=1.06, xanchor="left", x=0))
fig.show()